In [41]:
from openai import OpenAI
import os
import json
from dotenv import load_dotenv

load_dotenv()

def clean_json_tags(json_str):
    return json_str.replace("```json", "").replace("```", "").strip()

def write_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)


In [42]:
task = "请帮我调研一下web3.0的开发"

output_schema_plan_structure = [
    {
        "type": "object",
        "properties": {
            "step_number": {"type": "interger"},
            "description": {"type": "string"}
        }
    }
]

SYSTEM_PROMPT = f"""你是一个专业的任务规划专家。你的目标是将用户给定的复杂请求分解成一系列清晰、有序且可独立执行的子步骤。

**规则：**
1.  每一步都应该是一个具体的、可操作的任务描述，避免模糊不清的指令。
2.  确保所有步骤的逻辑顺序是正确的，并且一步接一步能够最终达成用户请求的目标。
3.  不要跳过任何必要的中间步骤。
4.  不要包含多余的、与最终目标无关的步骤。
5.  你的输出应该是一个编号的步骤列表，每一步以“步骤N:”开头。


请按照以下 JSON 格式定义的格式输出结果：

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema_plan_structure, indent=2)}
</OUTPUT JSON SCHEMA>

step_number和description属性将用于后续的深入研究。
请确保输出是一个符合上述 JSON 格式定义的 JSON 对象。
只返回 JSON 对象，不要附加任何解释或额外文本。
请始终使用中文
"""

In [43]:
llm = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"), 
    base_url="https://api.deepseek.com"
    )

llm_response = llm.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": task}
    ],
    stream=False
    )


plan = clean_json_tags(llm_response.choices[0].message.content)


write_json(plan, "plan.md")


In [51]:
# reflection_prompt = {
#     "任务": "反思",
#     "上下文": {
#         "用户询问": task,
#         "生成的计划": plan
#     },
#     "指令": [
#         "审查生成的计划否有改进空间",
#         "审查对于用户的询问，生成的计划是否有遗漏的点",
#         "检查计划是否高效"
#     ],
#     "输出格式": [
#         "步骤1: [修改后的步骤1]",
#         "步骤2: [修改后的步骤2]",
#         "...",
#         "步骤N: [修改后的步骤N]"
#     ]
# }

reflection_prompt = f"""
你是一个专业的任务规划专家。你的目标是审查生成的计划否有改进空间，审查对于用户的询问，生成的计划是否有遗漏的点，检查计划是否高效。

用户询问: {task}
生成的计划: {plan}


请把修改后的最终计划按照以下 JSON 格式定义的格式输出结果：

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema_plan_structure, indent=2)}
</OUTPUT JSON SCHEMA>

请确保输出是一个符合上述 JSON 格式定义的 JSON 对象。
只返回 JSON 对象，不要附加任何解释或额外文本。
请始终使用中文
"""

In [52]:
reflection_task = json.dumps(reflection_prompt, ensure_ascii=False)

llm_response = llm.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": task},
        {"role": "assistant", "content": plan},
        {"role": "user", "content": reflection_task}
    ],
    stream=False
    )

reflection = llm_response.choices[0].message.content

write_json(reflection, "final_plan.md")

In [50]:
print(plan)

[
  {
    "step_number": 1,
    "description": "确定Web3.0的定义和核心技术组成，包括区块链、智能合约、去中心化应用（DApps）等。"
  },
  {
    "step_number": 2,
    "description": "收集并分析当前主流的Web3.0开发平台和工具，如以太坊、Polkadot、Solana等。"
  },
  {
    "step_number": 3,
    "description": "调研Web3.0开发所需的主要编程语言和框架，例如Solidity、Rust、Substrate等。"
  },
  {
    "step_number": 4,
    "description": "研究Web3.0开发中的常见用例和应用场景，如DeFi、NFT、DAO等。"
  },
  {
    "step_number": 5,
    "description": "分析Web3.0开发中的主要挑战和限制，如可扩展性、安全性、用户体验等。"
  },
  {
    "step_number": 6,
    "description": "总结当前Web3.0开发社区和资源，包括开源项目、论坛、教程等。"
  },
  {
    "step_number": 7,
    "description": "撰写调研报告，汇总所有收集到的信息，并提供未来发展的建议。"
  }
]


In [ ]:
print(reflection)